In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras_self_attention import SeqSelfAttention
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



Using TensorFlow backend.


id,sentence1,sentence2,label
1,"In Paris , in October 1560 , he secretly met the English ...","In October 1560 , he secretly met with the English ambas ...",0
2,The NBA season of 1975 -- 76 was the 30th season of the ...,The 1975 -- 76 season of the National Basketball Associa ...,1
3,"There are also specific discussions , public profile deb ...","There are also public discussions , profile specific dis ...",0
4,"When comparable rates of flow can be maintained , the re ...",The results are high when comparable flow rates can be m ...,1
5,It is the seat of Zerendi District in Akmola Region .,It is the seat of the district of Zerendi in Akmola region .,1


id,sentence1,sentence2,label
1,Bradd Crellin represented BARLA Cumbria on a tour of Aus ...,Bradd Crellin also represented BARLA Great Britain on a ...,0
2,They were there to enjoy us and they were there to pray ...,They were there for us to enjoy and they were there for ...,1
3,"After the end of the war in June 1902 , Higgins left Sou ...","In August , after the end of the war in June 1902 , Higg ...",1
4,From the merger of the Four Rivers Council and the Audub ...,Shawnee Trails Council was formed from the merger of the ...,1
5,The group toured extensively and became famous in Israel ...,The group toured extensively and was famous in Israel an ...,1


id,sentence1,sentence2,label
1,"This was a series of nested angular standards , so that ...","This was a series of nested polar scales , so that measu ...",0
2,His father emigrated to Missouri in 1868 but returned wh ...,"His father emigrated to America in 1868 , but returned w ...",0
3,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",1
4,"Steiner argued that , in the right circumstances , the s ...",Steiner held that the spiritual world can be researched ...,0
5,"Luciano Williames Dias ( born July 25 , 1970 ) is a Braz ...",Luciano Williames Dias ( born 25 July 1970 ) is a former ...,0


In [18]:
ppdb_Tb=Table.read_table("dataset/ppdb/sanitized.tsv")

In [20]:
test_Tb=Table.read_table("dataset/PAWS_Wiki/test.tsv")

In [21]:

test_id=test_Tb.column("id")
test_row=test_Tb.num_rows
test_sen1=test_Tb.column("sentence1")
test_sen2=test_Tb.column("sentence2")

test_label=test_Tb.column("label")

phrase = ppdb_Tb.column("PHRASE")
paraphrase=ppdb_Tb.column("PARAPHRASE")   #ppdb


In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(test_id)):   #index of PAWS_Wiki training set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in test_sen1[x] and paraphrase[z] in test_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, test_sen1[x])
            sent2=np.append(sent2, test_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, test_label[x])

 together with 
 together with 
 's used 
 's used 
 along with 
 the southern part 
 just outside 
 first time 
 first time 
 economic reform 
 economic reform 
 left side 
 the left side 
 left side 
 newly created 
 members of the 
 published in 
 published in 
 other two 
 after only 
 along with 
 in collaboration with 
 collaboration with the 
 collaboration with 
 collaboration with 
 in collaboration 
 in collaboration with the 
 in collaboration with the 
 in collaboration 
 collaboration with 
 almost always 
 almost always 
 police officers were 
 officers were 
 are totally 
 are totally 
 are totally 
 are totally 
 's history 
 shortly thereafter 
 shortly thereafter 
 years old 
 17 years 
 most important 
 most important 
 ever recorded 
 recorded 
 an integral part 
 northern part 
 can also be used 
 can also be used 
 can also 
 newly created 
 members of the 
 said that his 
 most successful 
 members of the 
 tell him 
 high school 
 high school 
 second round 
 , 

In [29]:
phrase_dataset = Table().with_column("TEST_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [30]:
phrase_dataset.show(5)

TEST_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label
2,786299,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",together with,along with,1
2,1.5301e+06,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",together with,along with,1
12,1.00571e+06,The spectral levels of light that can be measured by pla ...,The spectral light levels that can be measured by plants ...,'s used,is used,1
12,1.00571e+06,The spectral levels of light that can be measured by pla ...,The spectral light levels that can be measured by plants ...,'s used,is used,1
24,903367,"In January 2011 , FIBA Asia deputy secretary general Man ...","In January 2011 , the Deputy Secretary General of FIBA A ...",along with,together with,0


In [31]:

sent1=phrase_dataset.column("sent1")
sent2=phrase_dataset.column("sent2")
para1=phrase_dataset.column("paraphrase1")
para2=phrase_dataset.column("paraphrase2")

gg = [x.split() for x in sent1]
hh = [x.split() for x in sent2]
ii = [x.split() for x in para1]
jj = [x.split() for x in para2]



22

In [32]:
labels1=[]
for x in range(len(gg)):
    labels1.append(spit_label(gg[x], ii[x], len(ii[x]),len(ii[x]), []))



In [33]:
labels2=[]
for x in range(len(hh)):
    labels2.append(spit_label(hh[x], jj[x], len(jj[x]),len(jj[x]), []))



In [34]:
joined1=[]
for x in labels1:
    joined1.append(" ".join(x))

In [35]:
joined2=[]
for x in labels2:
    joined2.append(" ".join(x))

In [36]:
phrase_dataset=phrase_dataset.with_columns("labels1", joined1, "labels2", joined2)


In [37]:
phrase_dataset

TEST_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label,labels1,labels2
2,786299,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",together with,along with,1,O O O O O O O O O O O O O O O O O O B E O O O O O O O,O O O O O O O O O O O B E O O O O O O O O O
2,1.5301e+06,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",together with,along with,1,O O O O O O O O O O O O O O O O O O B E O O O O O O O,O O O O O O O O O O O B E O O O O O O O O O
12,1.00571e+06,The spectral levels of light that can be measured by pla ...,The spectral light levels that can be measured by plants ...,'s used,is used,1,O O O O O O O O O O O O O O O O O O O O O O O B E O O O,O O O O O O O O O O O O O O O O O O O O O B E O O O
12,1.00571e+06,The spectral levels of light that can be measured by pla ...,The spectral light levels that can be measured by plants ...,'s used,is used,1,O O O O O O O O O O O O O O O O O O O O O O O B E O O O,O O O O O O O O O O O O O O O O O O O O O B E O O O
24,903367,"In January 2011 , FIBA Asia deputy secretary general Man ...","In January 2011 , the Deputy Secretary General of FIBA A ...",along with,together with,0,O O O O O O O O O O O O B E O O O O O O O O,O O O O O O O O O O O O O O O O O O B E O O O O O O O
37,478094,"Located in the southern part of the county , it lies in ...","It is located in the southeastern part of the county , i ...",the southern part,southern,0,O O B I E O O O O O O O O O O O O O O,O O O O O O O O O O O O O S O O O O O
40,684109,After selling his house on 30 Harare drive to the Canadi ...,After selling his house on 30 Harare ride to the Canadia ...,just outside,outside,1,O O O O O O O O O O O O O O O O B E O O O O,O O O O O O O O O O O O O O O O O S O O O O
62,88753,"For the first time in league history , Gary Gait of Roch ...","For the first time in the history of the league , Paul G ...",first time,first,0,O O B E O O O O O O O O O O O O O O O O O O,O O S O O O O O O O O O O O O O O O O O O O O O O
62,646502,"For the first time in league history , Gary Gait of Roch ...","For the first time in the history of the league , Paul G ...",first time,first,0,O O B E O O O O O O O O O O O O O O O O O O,O O S O O O O O O O O O O O O O O O O O O O O O O
74,1.25578e+06,"The death of Mao Zedong in 1976 , however , brought econ ...","The death of Mao Zedong in 1976 , however , brought econ ...",economic reform,reform,0,O O O O O O O O O O O B E O O O O O O O O O O O O O O O O O,O O O O O O O O O O O O S O O O O O O O O O O O O O O O O O


In [38]:
phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_TEST.tsv", sep='\t', index=False)

In [2]:
    def spit_label(split_sent, para_split, ori_length, length, listy):
       
    
        if para_split[0]==split_sent[0]:
            
            if ori_length==1:
                listy.append("S")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy    
            
            elif ori_length==length:
                listy.append("B")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
            elif length==1:
                listy.append("E")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy
            
            else:
                listy.append("I")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
        else:
            listy.append("O")
            return spit_label(split_sent[1:], para_split, ori_length, length, listy)

In [88]:
bbb="".join("he ")

In [89]:
bbb

'he '